In [ ]:
%%writefile tiny.c
int main(void) {
    return 42;
}

In [ ]:
!gcc -o tiny_binary tiny.c
!file tiny_binary
!ls -l tiny_binary

In [ ]:
import base64

with open("tiny_binary", "rb") as f:
    b64 = base64.b64encode(f.read()).decode("ascii")

print(b64[:200], "...")   # preview
print("Length:", len(b64))


In [ ]:
import base64

with open("tiny_binary_copy", "wb") as f:
    f.write(base64.b64decode(b64))

!file tiny_binary_copy

In [ ]:
!objdump -d tiny_binary | head -n 80

In [ ]:
!nm -n tiny_binary | head -n 50

In [ ]:
!nm -n tiny_binary | grep " main"


In [ ]:
!objdump -d tiny_binary | sed -n '1,200p'


In [ ]:
!objdump -d -Mintel tiny_binary | sed -n '1,120p'


In [ ]:
!pip -q install angr


In [ ]:
import angr
import networkx as nx
import matplotlib.pyplot as plt

proj = angr.Project("tiny_binary", auto_load_libs=False)
cfg = proj.analyses.CFGFast()

print("Functions found:", len(cfg.kb.functions))

# Try main; if missing, pick first discovered function
func = cfg.kb.functions.function(name="main") or next(iter(cfg.kb.functions.values()))
print("Using function:", func.name, hex(func.addr))

G = nx.DiGraph(func.graph)  # CFG graph for that function

plt.figure(figsize=(10, 7))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, node_size=140, with_labels=False)
plt.title(f"CFG for {func.name} @ {hex(func.addr)}")
plt.show()


In [ ]:
cg = cfg.kb.callgraph  # networkx DiGraph
print("Call graph nodes:", cg.number_of_nodes())
print("Call graph edges:", cg.number_of_edges())

# Print a few edges (addresses)
for u, v in list(cg.edges())[:30]:
    print(hex(u), "->", hex(v))


In [ ]:
!gcc -O0 -fno-pie -no-pie -o tiny_binary tiny.c
!objdump -d -Mintel tiny_binary | grep -n "<main>" -n
